<h1> Scaling up ML using Cloud ML Engine </h1>

In this notebook, we take a previously developed TensorFlow model to predict taxifare rides and package it up so that it can be run in Cloud MLE. For now, we'll run this on a small dataset. The model that was developed is rather simplistic, and therefore, the accuracy of the model is not great either.  However, this notebook illustrates *how* to package up a TensorFlow model to run it within Cloud ML. 

Later in the course, we will look at ways to make a more effective machine learning model.

<h2> Environment variables for project and bucket </h2>

Note that:
<ol>
<li> Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos </li>
<li> Cloud training often involves saving and restoring model files. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available). A common pattern is to prefix the bucket name by the project id, so that it is unique. Also, for cost reasons, you might want to use a single region bucket. </li>
</ol>
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [7]:
import os
output = os.popen("gcloud config get-value project").readlines()
project_name = output[0][:-1]

# change these to try this notebook out
PROJECT = project_name
BUCKET = project_name
BUCKET = project_name #BUCKET.replace("qwiklabs-gcp-", "inna-bckt-")
REGION = 'europe-west1'  ## note: Cloud ML Engine not availabe in europe-west3!

print(PROJECT)
print(BUCKET)
print("gsutil mb -l {0} gs://{1}".format(REGION, BUCKET))

## set config for gcp config: [[?]]
print(os.popen("gcloud config set project $PROJECT").readlines())
print(os.popen("gcloud config set compute/region $REGION").readlines())

qwiklabs-gcp-7690f94228f38b0b
qwiklabs-gcp-7690f94228f38b0b
gsutil mb -l europe-west1 gs://qwiklabs-gcp-7690f94228f38b0b
[]
[]


In [8]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'  # Tensorflow version

In [9]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


Allow the Cloud ML Engine service account to read/write to the bucket containing training data.

In [10]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print(response['serviceAccount'])")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-878188778647@cloud-ml.google.com.iam.gserviceaccount.com to access files in qwiklabs-gcp-7690f94228f38b0b


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   235    0   235    0     0    244      0 --:--:-- --:--:-- --:--:--   244
No changes to gs://qwiklabs-gcp-7690f94228f38b0b/
Encountered a problem: CommandException: No URLs matched: gs://qwiklabs-gcp-7690f94228f38b0b/*
No changes to gs://qwiklabs-gcp-7690f94228f38b0b/


<h2> Packaging up the code </h2>

Take your code and put into a standard Python package structure.  <a href="taxifare/trainer/model.py">model.py</a> and <a href="taxifare/trainer/task.py">task.py</a> containing the Tensorflow code from earlier (explore the <a href="taxifare/trainer/">directory structure</a>).

In [10]:
%bash
# TODO: Make sure that model is complete and has no remaining TODOs
grep TODO taxifare/trainer/*.py

taxifare/trainer/model.py:# TODO: Add input function
taxifare/trainer/model.py:# TODO: Create tf.estimator.DNNRegressor train and evaluate function passing args['parsed_argument'] from task.py


<h2> Find absolute paths to your data </h2>

Note the absolute paths below. /content is mapped in Datalab to where the home icon takes you

In [11]:
%bash
echo $PWD
rm -rf $PWD/taxi_trained
head -1 $PWD/taxi-train.csv
head -1 $PWD/taxi-valid.csv

/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs
12.0,-73.987625,40.750617,-73.971163,40.78518,1,0
6.0,-74.013667,40.713935,-74.007627,40.702992,2,0


<h2> Running the Python module from the command-line </h2>

<h4> Monitor using Tensorboard </h4>

In [13]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./taxi_trained')

TensorBoard was started successfully with pid 4654. Click here to access it.

4654

In [17]:
%bash
rm -rf taxifare.tar.gz taxi_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
python -m trainer.task \
   --train_data_paths="${PWD}/taxi-train*" \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --output_dir=${PWD}/taxi_trained \
   --train_steps=100 --job-dir=./tmp

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3a398b3810>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi_trained', '_global_id

In [18]:
%bash
ls $PWD/taxi_trained/export/exporter/

1549013827


In [19]:
%writefile ./test.json
{"pickuplon": -73.885262,"pickuplat": 40.773008,"dropofflon": -73.987232,"dropofflat": 40.732403,"passengers": 2}

Writing ./test.json


In [20]:
%bash
model_dir=$(ls ${PWD}/taxi_trained/export/exporter)
gcloud ml-engine local predict \
    --model-dir=${PWD}/taxi_trained/export/exporter/${model_dir} \
    --json-instances=./test.json

PREDICTIONS
[10.85594367980957]


  from ._conv import register_converters as _register_converters



<h2> Running locally using gcloud </h2>

In [21]:
%bash
rm -rf taxifare.tar.gz taxi_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   -- \
   --train_data_paths=${PWD}/taxi-train.csv \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/taxi_trained 

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_paths=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-train.csv', u'--eval_data_paths=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-valid.csv', u'--train_steps=1000', u'--output_dir=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi_trained'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_trai

When I ran it (due to random seeds, your results will be different), the ```average_loss``` (Mean Squared Error) on the evaluation dataset was 187, meaning that the RMSE was around 13.

In [22]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

Stopped TensorBoard with pid 4635
Stopped TensorBoard with pid 4654


If the above step (to stop TensorBoard) appears stalled, just move on to the next step. You don't need to wait for it to return.

In [23]:
!ls $PWD/taxi_trained

checkpoint				     model.ckpt-1000.index
eval					     model.ckpt-1000.meta
events.out.tfevents.1549013915.b458f27b4ec5  model.ckpt-1.data-00000-of-00001
export					     model.ckpt-1.index
graph.pbtxt				     model.ckpt-1.meta
model.ckpt-1000.data-00000-of-00001


<h2> Submit training job using gcloud </h2>

First copy the training data to the cloud.  Then, launch a training job.

After you submit the job, go to the cloud console (http://console.cloud.google.com) and select <b>Machine Learning | Jobs</b> to monitor progress.  

<b>Note:</b> Don't be concerned if the notebook stalls (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud.  Use the Cloud Console link (above) to monitor the job.

In [24]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/taxifare/smallinput/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/taxifare/smallinput/

qwiklabs-gcp-7690f94228f38b0b


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-eval.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-valid.csv [Content-Type=text/csv]...
/ [6/6 files][664.2 KiB/664.2 KiB] 100% Done                                    
Operation completed over 6 objects/664.2 KiB.                                

In [25]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/smallinput/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained europe-west1 lab3a_190201_094123
jobId: lab3a_190201_094123
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [lab3a_190201_094123] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe lab3a_190201_094123

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs lab3a_190201_094123


Don't be concerned if the notebook appears stalled (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud. 

<b>Use the Cloud Console link to monitor the job and do NOT proceed until the job is done.</b>

In [26]:
!gcloud ml-engine jobs describe lab3a_190201_094123

createTime: '2019-02-01T09:41:29Z'
etag: Baw3HdbsQmw=
jobId: lab3a_190201_094123
state: PREPARING
trainingInput:
  args:
  - --train_data_paths=gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi-train*
  - --eval_data_paths=gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi-valid*
  - --output_dir=gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained
  - --train_steps=10000
  jobDir: gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained
  packageUris:
  - gs://qwiklabs-gcp-7690f94228f38b0b/lab3a_190201_094123/c8ae3805ddba8293ac3e35556ffdb7457642d4bed361800c757623d40b8e18d1/trainer-0.0.0.tar.gz
  pythonModule: trainer.task
  region: europe-west1
  runtimeVersion: '1.8'
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/lab3a_190201_094123?project=qwiklabs-gcp-7690f94228f38b0b

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2Flab3a_190201_094123&project=qwikl

<h2> Deploy model </h2>

Find out the actual name of the subdirectory where the model is stored and use it to deploy the model.  Deploying model will take up to <b>5 minutes</b>.

In [27]:
%bash
gsutil ls gs://${BUCKET}/taxifare/smallinput/taxi_trained/export/exporter

gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained/export/exporter/
gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained/export/exporter/1549014258/
gs://qwiklabs-gcp-7690f94228f38b0b/taxifare/smallinput/taxi_trained/export/exporter/1549014719/


In [28]:
%bash
MODEL_NAME="taxifare"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/taxifare/smallinput/taxi_trained/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Run these commands one-by-one (the very first time, you'll create a model and then create a version)


Created ml engine model [projects/qwiklabs-gcp-7690f94228f38b0b/models/taxifare].
Creating version (this might take a few minutes)......
.........................................................................................................................................................................................................................................................................................................................................done.


<h2> Prediction </h2>

In [29]:
%bash
gcloud ml-engine predict --model=taxifare --version=v1 --json-instances=./test.json

PREDICTIONS
[11.100281715393066]


In [30]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'pickuplon': -73.885262,
        'pickuplat': 40.773008,
        'dropofflon': -73.987232,
        'dropofflat': 40.732403,
        'passengers': 2,
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'taxifare', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

response={u'predictions': [{u'predictions': [11.100281715393066]}]}


<h2> Train on larger dataset </h2>

I have already followed the steps below and the files are already available. <b> You don't need to do the steps in this comment. </b> In the next chapter (on feature engineering), we will avoid all this manual processing by using Cloud Dataflow.

Go to http://bigquery.cloud.google.com/ and type the query:
<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  'nokeyindata' AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  AND ABS(HASH(pickup_datetime)) % 1000 == 1
</pre>

Note that this is now 1,000,000 rows (i.e. 100x the original dataset).  Export this to CSV using the following steps (Note that <b>I have already done this and made the resulting GCS data publicly available</b>, so you don't need to do it.):
<ol>
<li> Click on the "Save As Table" button and note down the name of the dataset and table.
<li> On the BigQuery console, find the newly exported table in the left-hand-side menu, and click on the name.
<li> Click on "Export Table"
<li> Supply your bucket name and give it the name train.csv (for example: gs://cloud-training-demos-ml/taxifare/ch3/train.csv). Note down what this is.  Wait for the job to finish (look at the "Job History" on the left-hand-side menu)
<li> In the query above, change the final "== 1" to "== 2" and export this to Cloud Storage as valid.csv (e.g.  gs://cloud-training-demos-ml/taxifare/ch3/valid.csv)
<li> Download the two files, remove the header line and upload it back to GCS.
</ol>

<p/>
<p/>

<h2> Run Cloud training on 1-million row dataset </h2>

This took 60 minutes and uses as input 1-million rows.  The model is exactly the same as above. The only changes are to the input (to use the larger dataset) and to the Cloud MLE tier (to use STANDARD_1 instead of BASIC -- STANDARD_1 is approximately 10x more powerful than BASIC).  At the end of the training the loss was 32, but the RMSE (calculated on the validation dataset) was stubbornly at 9.03. So, simply adding more data doesn't help.

In [ ]:
%%bash

XXXXX  this takes 60 minutes. if you are sure you want to run it, then remove this line.

OUTDIR=gs://${BUCKET}/taxifare/ch3/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
CRS_BUCKET=cloud-training-demos # use the already exported data
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/train.csv" \
   --eval_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/valid.csv"  \
   --output_dir=$OUTDIR \
   --train_steps=100000

## Challenge Exercise

Modify your solution to the challenge exercise in d_trainandevaluate.ipynb appropriately. Make sure that you implement training and deployment. Increase the size of your dataset by 10x since you are running on the cloud. Does your accuracy improve?

In [31]:
import datalab.bigquery as bq
import tensorflow as tf
import numpy as np
import shutil
print(tf.__version__)

from google.datalab.ml import TensorBoard

import math
import pandas as pd

def gen_data(n):
  r = np.random.uniform(.5, 2, n)
  h = np.random.uniform(.5, 2, n)
  v = r ** 2 * h * math.pi
  dat = pd.DataFrame({
    'r': np.round(r, 1),
    'h': np.round(h, 1),
    'v': np.round(v, 1)
  })
  return dat


dat = gen_data(5000)
dat_eval = gen_data(1000)
dat_test = gen_data(1000)

dat.head(n = 2)


1.8.0


,h,r,v
0,1.7,1.6,13.3
1,1.1,1.9,12.4


In [38]:
## write data to file:
dat.to_csv('dat-cyl-train.csv', header = False, index = False)
dat_eval.to_csv('dat-cyl-eval.csv', header = False, index = False)
dat_test.to_csv('dat-cyl-test.csv', header = False, index = False)

In [40]:
!ls -l dat-cyl*.csv

-rw-r--r-- 1 root root 12224 Feb  1 10:23 dat-cyl-eval.csv
-rw-r--r-- 1 root root 12249 Feb  1 10:23 dat-cyl-test.csv
-rw-r--r-- 1 root root 61228 Feb  1 10:23 dat-cyl-train.csv


In [41]:
%bash
echo $PWD
rm -rf $PWD/cyl_trained
head -1 $PWD/dat-cyl-train.csv
head -1 $PWD/dat-cyl-eval.csv

/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs
1.7,1.6,13.3
1.2,1.1,4.7


## Running locally using Python

In [42]:
%bash
rm -rf labs/cyl.tar.gz labs/cyl_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/cyl
python -m trainer.task \
   --train_data_paths="${PWD}/dat-cyl-train*" \
   --eval_data_paths=${PWD}/dat-cyl-eval.csv  \
   --output_dir=${PWD}/labs/cyl_trained \
   --train_steps=1000 --job-dir=./tmp

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f50bebf3810>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/labs/cyl_trained', '_globa

In [43]:
%bash
ls $PWD/taxi_trained/export/exporter/

1549013939


In [44]:
%writefile ./test.json
{"h": 1.0,"r": 2.0}

Overwriting ./test.json


In [50]:
%bash
model_dir=$(ls ${PWD}/labs/cyl_trained/export/exporter)

In [52]:
%bash
model_dir=$(ls ${PWD}/labs/cyl_trained/export/exporter)

gcloud ml-engine local predict \
    --model-dir=${PWD}/labs/cyl_trained/export/exporter/${model_dir} \
    --json-instances=./test.json

PREDICTIONS
[12.165831565856934]


  from ._conv import register_converters as _register_converters



## Running locally using gcloud

In [53]:
%bash
rm -rf labs/cyl.tar.gz labs/cyl_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/cyl/trainer \
   -- \
   --train_data_paths=${PWD}/dat-cyl-train.csv \
   --eval_data_paths=${PWD}/dat-cyl-eval.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/labs/cyl_trained 

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--train_data_paths=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-train.csv', u'--eval_data_paths=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-eval.csv', u'--train_steps=1000', u'--output_dir=/content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/labs/cyl_trained'], u'job_name': u'trainer.task'}, u'task': {}}
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker

## Submit Training using gcloud

In [54]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/cyl/smallinput/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/cyl/smallinput/

qwiklabs-gcp-7690f94228f38b0b


CommandException: 1 files/objects could not be removed.
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/dat-cyl-eval.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-test.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-train.csv [Content-Type=text/csv]...
Copying file:///content/datalab/gcp-ml-01-ml-with-tf-on-gcp/03-intro-to-tensorflow/labs/taxi-valid.csv [Content-Type=text/csv]...
/ [6/6 files][664.3 KiB/664.3 KiB] 100% Done                                    
Operation completed over 6 objects/664.3 KiB.                                

In [55]:
%%bash
OUTDIR=gs://${BUCKET}/cyl/smallinput/cyl_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/cyl/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/cyl/smallinput/dat-cyl-train*" \
   --eval_data_paths="gs://${BUCKET}/cyl/smallinput/dat-cyl-eval*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

gs://qwiklabs-gcp-7690f94228f38b0b/cyl/smallinput/cyl_trained europe-west1 lab3a_190201_103842
jobId: lab3a_190201_103842
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [lab3a_190201_103842] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe lab3a_190201_103842

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs lab3a_190201_103842


Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License